In [1]:
from pathlib import Path
import pickle

import numpy as np
import numpy.random as nr

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, f1_score

# import xgboost as xgb
# from xgboost import XGBClassifier

In [2]:
# Print out packages versions
print(f'pandas version is: {pd.__version__}')
print(f'numpy version is: {np.__version__}')
print(f'matplotlib version is: {matplotlib.__version__}')
print(f'sklearn version is: {sklearn.__version__}')

pandas version is: 1.1.5
numpy version is: 1.19.5
matplotlib version is: 3.2.2
sklearn version is: 0.22.2.post1


# Helper functions

In [3]:
def replace_nan_inf(df, value=None):
    """
    Replace missing and infinity values.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with values to be replaced.

    value : int, float
        Value to replace any missing or numpy.inf values. Defaults to numpy.nan
    Returns
    -------
    pandas.DataFrame
        Dataframe with missing and infinity values replaced with -999.

    """
    if value is None:
        value = np.nan
    return df.replace(to_replace=[np.nan, np.inf, -np.inf],
                      value=value)


def shift_concat(df, periods=1, fill_value=None):
    """
    Build dataframe of shifted index.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns to be shifted.
    periods : int
        Number of periods to shift. Should be positive.
    fill_value : object, optional
        The scalar value to use for newly introduced missing values. Defaults
        to numpy.nan.

    Returns
    -------
    pandas.DataFrame
        Shifted dataframes concatenated along columns axis.

    Notes
    -------
    Based on Paulo Bestagini's augment_features_window from SEG 2016 ML
    competition.
    https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try01.ipynb

    Example
    -------
    Shift df by one period and concatenate.

    >>> df = pd.DataFrame({'gr': [1.1, 2.1], 'den': [2.1, 2.2]})
    >>> shift_concat(df)
        gr_shifted_1  den_shifted_1  gr   den  gr_shifted_-1   den_shifted_-1
    0      NaN            NaN        1.1  2.1      2.1             2.2
    1      1.1            2.1        2.1  2.2      NaN             NaN

    """
    if fill_value is None:
        fill_value = np.nan

    dfs = []
    for period in range(periods, -1*periods - 1, -1):

        if period == 0:
            dfs.append(df)
            continue

        df_shifted = df.shift(period, fill_value=fill_value)

        df_shifted.columns = [f'{col}_shifted_{str(period)}'
                              for col in df_shifted.columns]

        dfs.append(df_shifted)

    return pd.concat(dfs, axis=1)


def gradient(df, depth_col):
    """
    Calculate the gradient for all features along the provided `depth_col`
    column.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns to be used in the gradient calculation.
    depth_col : str
        Dataframe column name to be used as depth reference.

    Returns
    -------
    pandas.DataFrame
        Gradient of `df` along `depth_col` column. The depth column is not in
        the output dataframe.

    Notes
    -------
    Based on Paulo Bestagini's augment_features_window from SEG 2016 ML
    competition.
    https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try01.ipynb

    Example
    -------
    Calculate gradient of columns along `md`.

    >>> df = pd.DataFrame({'gr': [100.1, 100.2, 100.3],
                          'den': [2.1, 2.2, 2.3],
                          'md': [500, 500.5, 501]})
    >>> gradient(df, 'md')
        gr  den
    0  NaN  NaN
    1  0.2  0.2
    2  0.2  0.2

    """
    depth_diff = df[depth_col].diff()

    denom_zeros = np.isclose(depth_diff, 0)
    depth_diff[denom_zeros] = 0.001

    df_diff = df.drop(depth_col, axis=1)
    df_diff = df_diff.diff()

    # Add suffix to column names
    df_diff.columns = [f'{col}_gradient' for col in df_diff.columns]

    return df_diff.divide(depth_diff, axis=0)


def shift_concat_gradient(df, depth_col, well_col, cat_cols, periods=1, fill_value=None):
    """
    Augment features using `shif_concat` and `gradient`.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns to be augmented.
    depth_col : str
        Dataframe column name to be used as depth reference.
    well_col : str
        Dataframe column name to be used as well reference.
    cat_cols: list of str
        Encoded column names. The gradient calculation is not applied to these
        columns.
    periods : int
        Number of periods to shift. Should be positive.
    fill_value : object, optional
        The scalar value to use for newly introduced missing values. Defaults
        to numpy.nan.

    Returns
    -------
    pandas.DataFrame
        Augmented dataframe.

    Notes
    -------
    Based on Paulo Bestagini's augment_features_window from SEG 2016 ML
    competition.
    https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try01.ipynb

    Example
    -------
    Augment features of `df` by shifting and taking the gradient.

    >>> df = pd.DataFrame({'gr': [100.1, 100.2, 100.3, 20.1, 20.2, 20.3],
                          'den': [2.1, 2.2, 2.3, 1.7, 1.8, 1.9],
                           'md': [500, 500.5, 501, 1000, 1000.05, 1001],
                         'well': [1, 1, 1, 2, 2, 2]})
    >>> shift_concat_gradient(df, 'md', 'well', periods=1, fill_value=None)
        gr_shifted_1  den_shifted_1     gr    den  ...  well   md    gr_gradient  den_gradient
    0         NaN          NaN         100.1  2.1  ...   1   500.00        NaN           NaN
    1       100.1          2.1         100.2  2.2  ...   1   500.50   0.200000      0.200000
    2       100.2          2.2         100.3  2.3  ...   1   501.00   0.200000      0.200000
    3         NaN          NaN          20.1  1.7  ...   2  1000.00        NaN           NaN
    4        20.1          1.7          20.2  1.8  ...   2  1000.05   2.000000      2.000000
    5        20.2          1.8          20.3  1.9  ...   2  1001.00   0.105263      0.105263

    """
    # TODO 'Consider filling missing values created here with DataFrame.fillna'

    # Columns to apply gradient operation
    cat_cols.append(well_col)
    gradient_cols = [col for col in df.columns if col not in cat_cols]

    # Don't shift depth
    depth = df.loc[:, depth_col]

    grouped = df.groupby(well_col, sort=False)

    df_aug_groups = []
    for name, group in grouped:
        shift_cols_df = group.drop([well_col, depth_col], axis=1)

        group_shift = shift_concat(shift_cols_df,
                                   periods=periods,
                                   fill_value=fill_value)

        # Add back the well name and depth
        group_shift[well_col] = name
        group_shift[depth_col] = depth

        group_gradient = group.loc[:, gradient_cols]

        group_gradient = gradient(group_gradient, depth_col)

        group_aug = pd.concat([group_shift, group_gradient], axis=1)

        df_aug_groups.append(group_aug)

    return pd.concat(df_aug_groups)


def score(y_true, y_pred, scoring_matrix):
    """
    Competition scoring function.

    Parameters
    ----------
    y_true : pandas.Series
        Ground truth (correct) target values.
    y_pred : pandas.Series
        Estimated targets as returned by a classifier.
    scoring_matrix : numpy.array
        Competition scoring matrix.

    Returns
    ----------
    float
        2020 FORCE ML lithology competition custome score.

    """
    S = 0.0

    for true_val, pred_val in zip(y_true, y_pred):
        S -= scoring_matrix[true_val, pred_val]

    return S/y_true.shape[0]


def show_evaluation(y_true, y_pred):
    """
    Print model performance and evaluation.

    Parameters
    ----------
    y_true : pandas.Series
        Ground truth (correct) target values.
    y_pred: pandas.Series
        Estimated targets as returned by a classifier.

    """
    print(f'Competition score: {score(y_true, y_pred)}')
    print(f'Accuracy: {accuracy_score(y_true, y_pred)}')
    print(f'F1: {f1_score(y_true, y_pred, average="weighted")}')


def build_encoding_map(series):
    """
    Build dictionary with the mapping of series unique values to encoded
    values.

    Parameters
    ----------
    series : pandas.Series
        Series with categories to be encoded.

    Returns
    -------
    mapping : dict
        Dictionary mapping unique categories in series to encoded values.

    See Also
    --------
    label_encode_columns : Label encode a dataframe categorical columns.

    """
    unique_values = series.unique()

    mapping = {original: encoded
               for encoded, original in enumerate(unique_values)
               if original is not np.nan}

    return mapping


def label_encode_columns(df, cat_cols, mappings):
    """
    Label encode a dataframe categorical columns.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns to be encoded.
    cat_cols: list of str
        Column names to be encoded.
    mappings: dict of dict
        Dictionary containing a key-value mapping for each column to be
        encoded.

    Returns
    -------
    df : pandas.DataFrame
        Dataframe with the encoded columns added and the `cat_cols` removed.
    encoded_col_names: list of str
        Encoded column names.

    See Also
    --------
    build_encoding_map : Build a series encoding mapping.

    """
    df = df.copy()

    encoded_col_names = []
    for col in cat_cols:
        new_col = f'{col}_encoded'
        encoded_col_names.append(new_col)

        df[new_col] = df[col].map(mappings[col])

        df.drop(col, axis=1, inplace=True)

    return df, encoded_col_names

# Target maps

In [4]:
KEYS_TO_ORDINAL = {
    30000: 0,
    65030: 1,
    65000: 2,
    80000: 3,
    74000: 4,
    70000: 5,
    70032: 6,
    88000: 7,
    86000: 8,
    99000: 9,
    90000: 10,
    93000: 11
    }


KEYS_TO_LITHOLOGY = {30000: 'Sandstone',
                     65030: 'Sandstone/Shale',
                     65000: 'Shale',
                     80000: 'Marl',
                     74000: 'Dolomite',
                     70000: 'Limestone',
                     70032: 'Chalk',
                     88000: 'Halite',
                     86000: 'Anhydrite',
                     99000: 'Tuff',
                     90000: 'Coal',
                     93000: 'Basement'}

ORDINAL_TO_KEYS = {value: key for key, value in  KEYS_TO_ORDINAL.items()}

ORDINAL_TO_LITHOLOGY = {}
for ordinal_key, key in ORDINAL_TO_KEYS.items():
    ORDINAL_TO_LITHOLOGY[ordinal_key] = KEYS_TO_LITHOLOGY[key]

# Import data

First add a shortcut from the [google drive competition data location](https://drive.google.com/drive/folders/1GIkjq4fwgwbiqVQxYwoJnOJWVobZ91pL) to your own google drive. We will mount this drive, and access the data from it.

We will save the results to a diffent folder, where we have write access.

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
#should be edited to the present working directory of the user
data_source = '/content/drive/My Drive/FORCE 2020 lithofacies prediction from well logs competition/'

In [7]:
penalty_matrix = np.load(data_source + 'penalty_matrix.npy')

train = pd.read_csv(data_source + 'CSV_train.csv', sep=';')

test = pd.read_csv(data_source + 'CSV_test.csv', sep=';')

In [8]:
# Destination folder
out_data_dir = Path('/content/drive/My Drive/lith_pred/')

# Train model

In [61]:
class Model():
    '''
    class to lithology prediction
    '''
    def preprocess(self, df, cat_columns, mappings):
        # Grab model features
        drop_cols = [
                     'FORCE_2020_LITHOFACIES_CONFIDENCE',
                     'SGR',
                     'DTS',
                     'RXO',
                     'ROPA'
                     ]

        # Confirm drop columns are in df
        drop_cols = [col for col in drop_cols if col in df.columns]

        df.drop(drop_cols, axis=1, inplace=True)

        # Label encode
        df, encoded_col_names = label_encode_columns(df, cat_columns, mappings)
        
        # Augment using Bestagini's functions
        df_preprocesed = shift_concat_gradient(df,
                                               'DEPTH_MD',
                                               'WELL',
                                               encoded_col_names,
                                               periods=1,
                                               fill_value=None)
       
        return df_preprocesed

    def fit(self, X, y):
        model = RandomForestClassifier(n_estimators=100,
                                       n_jobs=-1,
                                       random_state=42,
                                       verbose=2)

        model.fit(X, y)

        return model

    def fit_predict(self, X_train, y_train, X_test, test_wells, save_filename):
        # Fit
        model = self.fit(X_train, y_train)

        model_proba = model.predict_proba(X_test)

        model_classes = [ORDINAL_TO_LITHOLOGY[lith] for lith in model.classes_]

        model_proba_df = pd.DataFrame(model_proba, columns=model_classes)

        model_proba_df['WELL'] = test_wells
        model_proba_df['DEPTH_MD'] = X_test['DEPTH_MD']

        # Create save directory if it doesn't exists
        if not save_filename.parent.is_dir():
            save_filename.parent.mkdir(parents=True)

        # Save models_proba to CSV
        model_proba_df.to_csv(save_filename, index=False)

        return model, model_proba_df

# Prepare train data

In [62]:
y = train['FORCE_2020_LITHOFACIES_LITHOLOGY']
X = train.drop('FORCE_2020_LITHOFACIES_LITHOLOGY', axis=1)

In [63]:
# Map lithology codes to scoring matrix index
y_train = y.map(KEYS_TO_ORDINAL)

In [64]:
model = Model()

In [65]:
X.columns

Index(['WELL', 'DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'GROUP', 'FORMATION',
       'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'SGR', 'NPHI', 'PEF',
       'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DCAL', 'DRHO', 'MUDWEIGHT', 'RMIC',
       'ROPA', 'RXO', 'FORCE_2020_LITHOFACIES_CONFIDENCE'],
      dtype='object')

In [66]:
cat_columns = ['GROUP', 'FORMATION']

train_mappings = {col: build_encoding_map(X[col]) for col in cat_columns}

X_train = model.preprocess(X, cat_columns, train_mappings)

In [67]:
X_train.drop('WELL', axis=1, inplace=True)

In [68]:
X_train.columns

Index(['X_LOC_shifted_1', 'Y_LOC_shifted_1', 'Z_LOC_shifted_1',
       'CALI_shifted_1', 'RSHA_shifted_1', 'RMED_shifted_1', 'RDEP_shifted_1',
       'RHOB_shifted_1', 'GR_shifted_1', 'NPHI_shifted_1', 'PEF_shifted_1',
       'DTC_shifted_1', 'SP_shifted_1', 'BS_shifted_1', 'ROP_shifted_1',
       'DCAL_shifted_1', 'DRHO_shifted_1', 'MUDWEIGHT_shifted_1',
       'RMIC_shifted_1', 'GROUP_encoded_shifted_1',
       'FORMATION_encoded_shifted_1', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI',
       'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS',
       'ROP', 'DCAL', 'DRHO', 'MUDWEIGHT', 'RMIC', 'GROUP_encoded',
       'FORMATION_encoded', 'X_LOC_shifted_-1', 'Y_LOC_shifted_-1',
       'Z_LOC_shifted_-1', 'CALI_shifted_-1', 'RSHA_shifted_-1',
       'RMED_shifted_-1', 'RDEP_shifted_-1', 'RHOB_shifted_-1',
       'GR_shifted_-1', 'NPHI_shifted_-1', 'PEF_shifted_-1', 'DTC_shifted_-1',
       'SP_shifted_-1', 'BS_shifted_-1', 'ROP_shifted_-1', 'DCAL_shifted_-1',
       'DRHO_shi

In [69]:
X_train = replace_nan_inf(X_train, value=-999.0)

# Prepare test data

In [70]:
test.shape

(136786, 23)

In [71]:
X_test = model.preprocess(test, cat_columns, train_mappings)
X_test.drop('WELL', axis=1, inplace=True)

In [72]:
X_test.shape

(136786, 83)

In [73]:
X_test.columns

Index(['X_LOC_shifted_1', 'Y_LOC_shifted_1', 'Z_LOC_shifted_1',
       'CALI_shifted_1', 'RSHA_shifted_1', 'RMED_shifted_1', 'RDEP_shifted_1',
       'RHOB_shifted_1', 'GR_shifted_1', 'NPHI_shifted_1', 'PEF_shifted_1',
       'DTC_shifted_1', 'SP_shifted_1', 'BS_shifted_1', 'ROP_shifted_1',
       'DCAL_shifted_1', 'DRHO_shifted_1', 'MUDWEIGHT_shifted_1',
       'RMIC_shifted_1', 'GROUP_encoded_shifted_1',
       'FORMATION_encoded_shifted_1', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI',
       'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS',
       'ROP', 'DCAL', 'DRHO', 'MUDWEIGHT', 'RMIC', 'GROUP_encoded',
       'FORMATION_encoded', 'X_LOC_shifted_-1', 'Y_LOC_shifted_-1',
       'Z_LOC_shifted_-1', 'CALI_shifted_-1', 'RSHA_shifted_-1',
       'RMED_shifted_-1', 'RDEP_shifted_-1', 'RHOB_shifted_-1',
       'GR_shifted_-1', 'NPHI_shifted_-1', 'PEF_shifted_-1', 'DTC_shifted_-1',
       'SP_shifted_-1', 'BS_shifted_-1', 'ROP_shifted_-1', 'DCAL_shifted_-1',
       'DRHO_shi

In [74]:
X_test = replace_nan_inf(X_test, value=-999.0)

# Fit and predict

In [75]:
save_filename = out_data_dir / 'model_proba/models_proba_most_columns_with_nans_no_split_rf.csv'

In [76]:
test_wells = test['WELL']

In [77]:
if save_filename.is_file():
    models_proba = pd.read_csv(save_filename)

else:
    model, model_proba = model.fit_predict(X_train, y_train, X_test, test_wells, save_filename)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  7.7min


building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 20.8min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.8s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    2.0s finished


# Explore models proba

In [78]:
model_proba.sample(10)

,Sandstone,Sandstone/Shale,Shale,Marl,Dolomite,Limestone,Chalk,Halite,Anhydrite,Tuff,Coal,Basement,WELL,DEPTH_MD
69315,0.03,0.20,0.71,0.02,0.00,0.04,0.00,0.0,0.0,0.00,0.00,0.00,29/3-1,4386.178001
10200,0.16,0.06,0.72,0.00,0.00,0.02,0.00,0.0,0.0,0.04,0.00,0.00,15/9-14,2032.396001
15921,0.02,0.02,0.02,0.31,0.00,0.45,0.17,0.0,0.0,0.00,0.00,0.01,15/9-14,2901.988001
80581,0.04,0.10,0.79,0.01,0.00,0.05,0.00,0.0,0.0,0.01,0.00,0.00,34/10-16 R,1936.992008
18417,0.13,0.44,0.38,0.02,0.00,0.01,0.00,0.0,0.0,0.00,0.01,0.01,15/9-14,3281.380001
64423,0.04,0.17,0.65,0.02,0.00,0.00,0.00,0.0,0.0,0.00,0.12,0.00,29/3-1,3641.834001
82684,0.02,0.04,0.86,0.03,0.00,0.05,0.00,0.0,0.0,0.00,0.00,0.00,34/10-16 R,2256.648008
32684,0.03,0.08,0.69,0.09,0.01,0.08,0.00,0.0,0.0,0.02,0.00,0.00,25/11-24,1925.635200
22553,0.04,0.07,0.80,0.01,0.02,0.04,0.00,0.0,0.0,0.02,0.00,0.00,25/10-10,1435.710400
49254,0.73,0.12,0.13,0.00,0.00,0.02,0.00,0.0,0.0,0.00,0.00,0.00,29/3-1,1323.682001


In [79]:
model_proba.describe()

,Sandstone,Sandstone/Shale,Shale,Marl,Dolomite,Limestone,Chalk,Halite,Anhydrite,Tuff,Coal,Basement,DEPTH_MD
count,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000
mean,0.173875,0.163459,0.493023,0.044218,0.004915,0.095390,0.004848,0.000166,0.000274,0.013203,0.006477,0.000151,2501.136889
std,0.231804,0.130350,0.269660,0.066050,0.009941,0.119065,0.032591,0.001491,0.004475,0.047096,0.039354,0.001244,1043.245788
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,227.296008
25%,0.030000,0.060000,0.260000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1707.948917
50%,0.070000,0.130000,0.540000,0.020000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2471.823595
75%,0.200000,0.230000,0.720000,0.060000,0.010000,0.110000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,3294.643006
max,1.000000,0.820000,1.000000,0.530000,0.190000,0.970000,0.440000,0.070000,0.340000,0.870000,0.960000,0.030000,5007.417976
